In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import datetime as dt

In [2]:
p1 = Path.cwd() / 'back_data'
url = p1 / 'kaggle-survey-2018.zip'
with zipfile.ZipFile(url) as z:
    print(z.namelist())
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    df = kag.iloc[1:, :]

['multipleChoiceResponses.csv', 'freeFormResponses.csv', 'SurveySchema.csv']


C:\Users\admin\AppData\Local\Temp\ipykernel_14724\2818711165.py:5: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))


In [8]:
df.loc[:, 'Q1'].value_counts()
q1 = (df.loc[:, 'Q1']
 .replace({'Prefer not to say':'Another', 'Prefer to self-describe':'Another'})
 .rename('Gender')
)
q1.value_counts()

Male       19430
Female      4010
Another      419
Name: Gender, dtype: int64

In [11]:
df.loc[:, 'Q2'].value_counts()
q2 = (df.loc[:, 'Q2']
 .str.slice(0, 2)
 .astype('int')
 .rename('Age')
)
q2.value_counts()


25    6159
22    5141
30    3776
18    3037
35    2253
40    1360
45     858
50     582
55     328
60     273
70      53
80      39
Name: Age, dtype: int64

In [39]:
df.loc[:, 'Q3'].value_counts()
q3 = (df.loc[:, 'Q3']
      .where(lambda ser: ser.isin(ser.value_counts().index[:7]), 'Other')
      .rename('Country')
)
q3.value_counts()

Other                       10733
United States of America     4716
India                        4417
China                        1644
Russia                        879
Brazil                        736
Germany                       734
Name: Country, dtype: int64

In [23]:
df.loc[:, 'Q4'].value_counts()
q4 = (df['Q4']
.replace({'Master’s degree': 18, 'Bachelor’s degree': 16, 'Doctoral degree': 20,
'Some college/university study without earning a bachelor’s degree': 13, 'Professional degree': 19,
'I prefer not to answer': np.nan, 'No formal education past high school': 12})
.fillna(11)
.rename('Edu')
)
q4.value_counts()

18.0    10855
16.0     7083
20.0     3357
13.0      967
11.0      766
19.0      599
12.0      232
Name: Edu, dtype: int64

In [58]:
df.loc[:, 'Q5'].value_counts()
q5 = (df.loc[:, 'Q5']
 .where(lambda ser: ser.isin(ser.value_counts().index[:4]), 'Other')
 .replace({'Computer science (software engineering, etc.)':'Computer', 'Engineering (non-computer focused)':'Engineering',
           'Mathematics or statistics':'Math/Stats', 'A business discipline (accounting, economics, finance, etc.)':'Business'})
.rename('Major')
)
q5.value_counts()

Computer       9430
Other          5983
Engineering    3705
Math/Stats     2950
Business       1791
Name: Major, dtype: int64

In [37]:
df.loc[:, 'Q6'].value_counts()
q6 = (df.loc[:, 'Q6']
 .replace({'Data Scientist':'DS/RS', 'Research Scientist':'DS/RS', 'Data Analyst':'DA/BA', 'Business Analyst':'DA/BA',
          'Data Engineer':'DE', 'DBA/Database Engineer':'DE'})
.where(lambda ser: ser.isin(ser.value_counts().index[:7]), 'Other')
.rename('Occupation')
)
q6.value_counts()

Other                5732
DS/RS                5326
Student              5253
Software Engineer    3130
DA/BA                2694
DE                    882
Not employed          842
Name: Occupation, dtype: int64

In [66]:
df.loc[:, 'Q7'].value_counts()
q7 = (df.loc[:, 'Q7']
 .replace({'I am a student':'STUDENT'})
.where(lambda ser: ser.isin(ser.value_counts().index[:7]), 'Other')
.rename('Field')
)
q7.value_counts()

Other                                     7813
Computers/Technology                      5584
STUDENT                                   4658
Academics/Education                       2749
Accounting/Finance                        1433
Online Service/Internet-based Services     871
Medical/Pharmaceutical                     751
Name: Field, dtype: int64

In [67]:
df.loc[:, 'Q8'].value_counts()
q8 = (df.loc[:, 'Q8']
 .str.replace(' ', '-')
 .str.split('-', expand=True).iloc[:, 0]
 .fillna(0)
 .astype('int')
 .rename('Experience')
)
q8.value_counts()

0     8656
1     3745
2     2577
5     2524
3     1751
10    1512
4     1488
15     854
20     384
30     197
25     171
Name: Experience, dtype: int64

In [63]:
df.loc[:, 'Q9'].value_counts()
q9 = (df.loc[:, 'Q9']
 .replace({'I do not wish to disclose my approximate yearly compensation':np.nan, '500,000+':'500'})
 .str.split('-', expand=True).iloc[:, 0]
 .apply(lambda ser: pd.to_numeric(ser, errors='ignore'))
 .mul(1000)
 .rename('Salary')
)
q9.value_counts()

0.0         4398
10000.0     1937
20000.0     1395
30000.0     1119
40000.0      965
50000.0      919
100000.0     843
60000.0      729
70000.0      677
90000.0      566
125000.0     533
80000.0      506
150000.0     457
200000.0     172
250000.0      75
500000.0      63
300000.0      52
400000.0      23
Name: Salary, dtype: int64

In [68]:
kag2 = pd.concat([q1, q2, q3, q4, q5, q6, q7, q8, q9], axis=1)
kag2

,Gender,Age,Country,Edu,Major,Occupation,Field,Experience,Salary
1,Female,45,United States of America,20.0,Other,Other,Other,0,NaN
2,Male,30,Other,16.0,Engineering,Other,Other,5,10000.0
3,Female,30,United States of America,18.0,Computer,DS/RS,STUDENT,0,0.0
4,Male,35,United States of America,18.0,Other,Not employed,Other,0,NaN
5,Male,22,India,18.0,Math/Stats,DA/BA,STUDENT,0,0.0
...,...,...,...,...,...,...,...,...,...
23855,Male,45,Other,20.0,Computer,Other,Computers/Technology,5,250000.0
23856,Female,25,Other,18.0,Engineering,Other,Other,0,NaN
23857,Male,22,Other,18.0,Computer,Software Engineer,Computers/Technology,0,10000.0
23858,Male,25,Other,11.0,Other,Other,Other,0,NaN


In [69]:
(kag2
 .groupby('Country')
 .apply(lambda group: group.Salary.corr(group.Experience))
)

Country
Brazil                      0.350281
China                       0.252835
Germany                     0.261992
India                       0.167262
Other                       0.302266
Russia                      0.198358
United States of America    0.353616
dtype: float64